<a href="https://colab.research.google.com/github/Ami03sa/Openai_projects-/blob/pdf_rag_system/Speak_with_any_PDF_file_PDF_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pdf ai machine using RAG


# 2. Libraries import

In [56]:
pip install openai

In [54]:
pip install PyPDF2


In [55]:
pip install pinecone

In [57]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI

# 3. Working with PDF files

![](https://miro.medium.com/v2/resize:fit:1400/1*FWwgOvUE660a04zoQplS7A.png)



### 3.1 Setting up API Key

In [68]:
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

### 3.2 Loading a PDF file




In [59]:
# Function to load a random PDF from a given directory
def load_pdf(file_name):
    # Read the PDF file
    pdf_file = open(file_name, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_content = ""
    # Extract text from each page
    for page in range(len(pdf_reader.pages)):
        text_content += pdf_reader.pages[page].extract_text()

    pdf_file.close()

    return text_content

In [60]:
# Function to chunk text by number of words or characters with a given size and overlap
def chunk_text(text, chunk_size=1500, chunk_overlap=100, by='word'):
    if by not in ['word', 'char']:
        raise ValueError("Invalid value for 'by'. Use 'word' or 'char'.")

    chunks = []
    if by == 'word':
        text = text.split()
    elif by == 'char':
        text = text

    current_chunk_start = 0
    while current_chunk_start < len(text):
        current_chunk_end = current_chunk_start + chunk_size
        if by == 'word':
            chunk = " ".join(text[current_chunk_start:current_chunk_end])
        else:
            chunk = text[current_chunk_start:current_chunk_end]

        chunks.append(chunk)
        current_chunk_start += (chunk_size - chunk_overlap)


    return chunks

In [61]:
pdf_loaded = load_pdf("state_of_ai_docs.pdf")

In [62]:
chunks = chunk_text(pdf_loaded, by='char')

In [63]:
len(chunks)

28

## 4. Building RAG system (Retrieval Augmented System)

In [70]:
# Pinecone init

from pinecone import Pinecone

pc = Pinecone(api_key="")
index = pc.Index("rag")

In [65]:
for i in range(len(chunks)):
    vector = client.embeddings.create(
        model="text-embedding-ada-002",
        input=chunks[i])
    vector = vector.data[0].embedding

    upsert_response = index.upsert(
    vectors=[
        (
         str(i),
         vector,
         {"chunk_content": chunks[i]}
        )
    ])

### 5. Building an interface to get proper answer based on the documentation


In [66]:

# Ask user a question
user_request = input("Ask some question regarding the document: ")

# Create embedding for the question
user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request
)

# Extract the embedding vector
user_vector = user_vector.data[0].embedding

# Query Pinecone index with keyword arguments
matches = index.query(
    vector=user_vector,
    top_k=1,
    include_metadata=True
)






Ask some question regarding the document: ai trends in 2023


In [67]:


messages = [
    {
        "role": "system",
        "content": """I want you to act as a support agent. Your name is "My Super Assistant".
        You will provide me with answers from the given info below.
        If the answer is not included, say exactly "Ooops! I don't know that." and stop after that.
        Refuse to answer any question not about the info. Never break character."""
    },
    {
        "role": "system",
        "content": f"Here is the info:\n{matches['matches'][0]['metadata']['chunk_content']}"
    },
    {
        "role": "user",
        "content": user_request
    }
]


chat_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0,
    max_tokens=400,
)


assistant_reply = chat_response.choices[0].message.content.strip()


if assistant_reply == "Ooops! I don't know that.":
    print("Assistant:", assistant_reply)
else:
    print("Assistant:", assistant_reply)
    print()
    print("Context: ", matches['matches'][0]['metadata']['chunk_content'])


Assistant: The state of AI in 2023 is marked by Generative AI’s breakout year. Organizations continue to see returns in the business areas in which they are using AI, and they plan to increase investment in the years ahead. A majority of respondents reported AI-related revenue increases within each business function using AI. More than two-thirds expect their organizations to increase their AI investment over the next three years.

Context:  duct and/or service development
Risk
Service operations
Strategy and corporate /f_inance
Supply chain management
Average across all functions≥20% 10/endash.case19% <10%
>10% 6/endash.case10% ≤5%4 10 26 40
14 41 55
4 11 26 41
4 9 18 31
5 13 26 44
8 12 34 54
7 5 19 31
9 24 33
4 10 28 42
9 34 17 60
16 16 34 66
8 19 38 65
12 25 24 61
13 16 35 64
10 14 33 57
10 16 32 58
23 3 30 56
6 18 35 59Organizations continue to see returns in the business areas in which they are using AI, and  
they plan to increase investment in the years ahead. We see a majority 